In [1]:
import numpy as np

In [2]:
fileName = r"E:\OneDrive - UNSW\PhD\Experiments\2021-06\openradar\data\adc_data.bin"
c = 3e8
band_width = 1798.92 * 1e6
range_res = c / (2 * band_width)

num_frame = 8
numADCSamples = 256
numADCBits = 16
numLanes = 4
isReal = 0
numRX = 4
numTX = 1
numChirps = 128
frame_period = 40 * 1e-3

In [3]:
# read .bin file
with open(fileName, "rb") as fid:
    adcData = np.array(np.frombuffer(fid.read(), dtype=np.int16))

# if 12 or 14 bits ADC per sample compensate for sign extension
if numADCBits != 16:
    l_max = 2**(numADCBits - 1) - 1
    adcData[adcData > l_max] -= 2**numADCBits

fileSize = adcData.shape[0]
print(fileSize)

# real data reshape, filesize = numADCSamples*numChirps
if isReal:
    numChirps = int(fileSize / numADCSamples / numRX)
    #create column for each chirp
    LVDS = np.reshape(adcData, (numADCSamples * numRX, numChirps),
                        order='F').transpose()
else:
    # for complex data
    # filesize = 2 * numADCSamples*numChirps
    numChirps = int(fileSize / 2 / numADCSamples / numRX)

    LVDS = np.zeros(int(fileSize / 2)).astype(complex)
    # combine real and imaginary part into complex data
    # read in file: 4I is followed by 4Q
    LVDS[::4] = adcData[::8] + complex(0, 1) * adcData[4::8]
    LVDS[1::4] = adcData[1::8] + complex(0, 1) * adcData[5::8]
    LVDS[2::4] = adcData[2::8] + complex(0, 1) * adcData[6::8]
    LVDS[3::4] = adcData[3::8] + complex(0, 1) * adcData[7::8]
    # create column for each chirp
    # each row is data from one chirp
    LVDS = np.reshape(LVDS, (numADCSamples * numRX, numChirps),
                        order='F').transpose()

# organize data per RX
adcData = np.zeros((numRX, numChirps * numADCSamples)).astype(complex)
for row in range(numRX):
    for i in range(numChirps):
        adcData[row, i * numADCSamples:(
            (i + 1) * numADCSamples)] = LVDS[i, row *
                                                numADCSamples:((row + 1) *
                                                            numADCSamples)]

13107200


In [4]:
CHIRP_LOOPS, NUM_RX, NUM_ADC_SAMPLES
np.reshape(adcData,128,4,256)

(4, 1638400)